In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasetss/order_products_prior_eda.csv
/kaggle/input/dataset/main_training_data_optimized.csv
/kaggle/input/datasetsss/orders_eda.csv
/kaggle/input/treee/other/default/1/final_training_pipeline.ipynb


Group-Based Data Splitting

In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc

# 1. Load data
features = pd.read_csv("/kaggle/input/dataset/main_training_data_optimized.csv")

# 2. Get unique users and split THEM (not the rows)
all_users = features['user_id'].unique()
train_users = np.random.choice(all_users, size=int(0.9 * len(all_users)), replace=False)

# 3. Create Train and Val masks based on User IDs
train_mask = features['user_id'].isin(train_users)

X_train = features[train_mask].drop(columns=['target', 'user_id', 'product_id'])
y_train = features[train_mask]['target']

X_val = features[~train_mask].drop(columns=['target', 'user_id', 'product_id'])
y_val = features[~train_mask]['target']

print(f"👥 Total Users: {len(all_users)} | Train Users: {len(train_users)} | Val Users: {len(all_users) - len(train_users)}")
del features
gc.collect()

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


👥 Total Users: 206209 | Train Users: 185588 | Val Users: 20621


0

LightGBM Dataset & Hyperparameters

In [5]:
# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

# Set optimized parameters
params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "verbosity": -1,
    "learning_rate": 0.020524119954249836,
    "num_leaves": 62,
    "max_depth": 7,
    "min_child_samples": 68,
    "feature_fraction": 0.7048511951639247,
    "bagging_fraction": 0.8216334105118741,
    "bagging_freq": 1,
    "lambda_l1": 0.5067095801454502,
    "lambda_l2": 4.219106216405748,
    "device": "gpu",
}

Training with Early Stopping

In [6]:
print("Starting Training...")

model = lgb.train(
    params,
    train_data,
    num_boost_round=1200,
    valid_sets=[train_data, val_data],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=50)
    ]
)

Starting Training...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 50 rounds
[50]	train's auc: 0.814686	valid's auc: 0.814569
[100]	train's auc: 0.816796	valid's auc: 0.81653
[150]	train's auc: 0.817897	valid's auc: 0.817538
[200]	train's auc: 0.818602	valid's auc: 0.818101
[250]	train's auc: 0.819081	valid's auc: 0.818422
[300]	train's auc: 0.819445	valid's auc: 0.818616
[350]	train's auc: 0.819766	valid's auc: 0.818694
[400]	train's auc: 0.820092	valid's auc: 0.818767
[450]	train's auc: 0.820389	valid's auc: 0.818821
[500]	train's auc: 0.820676	valid's auc: 0.818866
[550]	train's auc: 0.820952	valid's auc: 0.81891
[600]	train's auc: 0.821232	valid's auc: 0.81892
[650]	train's auc: 0.821475	valid's auc: 0.818958
[700]	train's auc: 0.821742	valid's auc: 0.818969
[750]	train's auc: 0.821995	valid's auc: 0.818962
Early stopping, best iteration is:
[714]	train's auc: 0.8218	valid's auc: 0.818975


Performance Evaluation (AUC)

In [7]:
from sklearn.metrics import roc_auc_score

# Generate predictions on the 10% validation set
val_preds = model.predict(X_val)

# Calculate final AUC
final_auc = roc_auc_score(y_val, val_preds)

print("\n" + "="*35)
print(f" MODEL PERFORMANCE")
print(f" VALIDATION AUC: {final_auc:.4f}")
print("="*35)

# Save for your GitHub repository
model.save_model('instacart_model.txt')
print(" Model exported as 'instacart_model.txt'")


 MODEL PERFORMANCE
 VALIDATION AUC: 0.8190
 Model exported as 'instacart_model.txt'
